In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

data = pd.read_csv("SolarPrediction.csv")

# Drop unnecessary columns
data.drop(['Date', 'Time'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Radiation', axis=1)
y = data['Radiation']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [2]:
from sklearn.neural_network import MLPRegressor

def create_ann():
    # Define the neural network structure
    ann = MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=200, random_state=42)
    return ann


In [3]:
import numpy as np
from tqdm import tqdm
from sklearn.exceptions import ConvergenceWarning
import warnings

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Define OIO parameters
num_particles = 30
max_iterations = 100
c1 = 0.1  # Acceleration constant
c2 = 0.1  # Damping constant
c3 = 0.1  # Step size constant

# Initialize particles
particles = np.random.uniform(-1, 1, (num_particles, X_train.shape[1]))

# Define the fitness function
def fitness_function(particle):
    ann = create_ann()
    ann.fit(X_train, y_train)
    predictions = ann.predict(X_test)
    mse = np.mean((predictions - y_test) ** 2)
    return mse

# Main OIO loop with progress bar
for iteration in tqdm(range(max_iterations), desc="OIO Training"):
    fitness_values = []
    for i in range(num_particles):
        fitness = fitness_function(particles[i])
        fitness_values.append(fitness)
        
        # Randomly generate a search direction
        search_direction = np.random.uniform(-1, 1, particles.shape[1])
        
        # Update particle position
        particles[i] = particles[i] + c1 * search_direction * fitness / (1 + c2 * np.linalg.norm(particles[i]))
        
        # Bound the particle position
        particles[i] = np.clip(particles[i], -1, 1)

# Select the best particle
best_particle_idx = np.argmin(fitness_values)
best_particle = particles[best_particle_idx]

# Train the final ANN with the best parameters
final_ann = create_ann()
final_ann.fit(X_train, y_train)

# Evaluate the final model
final_predictions = final_ann.predict(X_test)
final_mse = np.mean((final_predictions - y_test) ** 2)
print("Final MSE:", final_mse)


OIO Training:   1%|          | 1/100 [03:03<5:03:20, 183.85s/it]/home/samarth5854/anaconda3/envs/PsycheSail/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
